<a href="https://colab.research.google.com/github/cjh2001525/Intelligent-Weather-Analysis-Advisory-System/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai


In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration



Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here
# --- Optional installs (uncomment if needed in Colab / JupyterHub) ---
# !pip install pyinputplus==0.2.12

# --- Imports ---
import os
import re
import json
from typing import Dict, List, Any, Optional
from datetime import datetime, timezone
import requests
import matplotlib.pyplot as plt

# --- Global config ---
WTTR_URL = "https://wttr.in/{location}?format=j1"   # Using wttr.in as weather data source
REQUEST_TIMEOUT = 12
DEFAULT_FORECAST_DAYS = 5
UNITS = "metric"   # metric units (Celsius, km/h)

# Matplotlib style
plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["axes.grid"] = True

# --- Utility functions ---
def clamp_forecast_days(days: int) -> int:
    """Clamp forecast days to range [1, 5]."""
    try:
        d = int(days)
    except Exception:
        d = DEFAULT_FORECAST_DAYS
    return max(1, min(5, d))

def clean_location(location: str) -> str:
    """Clean up location string: trim and collapse spaces."""
    if not isinstance(location, str):
        return ""
    return re.sub(r"\s+", " ", location.strip())

def http_get_json(url: str) -> Dict[str, Any]:
    """
    Unified HTTP GET for JSON responses.
    Returns dict with either:
        {"ok": True, "data": <json>}
    or
        {"ok": False, "error": <message>}
    """
    try:
        r = requests.get(url, timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        return {"ok": True, "data": r.json()}
    except requests.RequestException as e:
        return {"ok": False, "error": str(e)}

def now_iso() -> str:
    """Return current UTC timestamp in ISO format."""
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")



## 🌤️ Weather Data Functions

In [1]:
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    # Clean and validate inputs
    location = clean_location(location)
    forecast_days = clamp_forecast_days(forecast_days)

    if not location:
        return {"ok": False, "error": "Invalid location string."}

    # Build API request
    url = WTTR_URL.format(location=location)
    result = http_get_json(url)
    if not result["ok"]:
        return {"ok": False, "error": result["error"]}

    data = result["data"]

    try:
        # Extract current condition
        current = data["current_condition"][0]
        current_weather = {
            "temperature": int(current.get("temp_C", 0)),
            "humidity": int(current.get("humidity", 0)),
            "condition": current["weatherDesc"][0]["value"],
            "wind_speed": int(current.get("windspeedKmph", 0)),
            "wind_direction": current.get("winddir16Point", "N/A"),
            "precipitation": float(current.get("precipMM", 0.0))
        }

        # Extract forecast days
        forecast_list = []
        for day in data["weather"][:forecast_days]:
            forecast_list.append({
                "date": day.get("date", ""),
                "min_temp": int(day.get("mintempC", 0)),
                "max_temp": int(day.get("maxtempC", 0)),
                # midday snapshot at index 4 (12:00)
                "condition": day["hourly"][4]["weatherDesc"][0]["value"],
                "precipitation_chance": int(day["hourly"][4].get("chanceofrain", 0))
            })

        # Return unified schema
        return {
            "ok": True,
            "location": location,
            "timestamp": now_iso(),
            "units": UNITS,
            "current": current_weather,
            "forecast": forecast_list
        }
    except Exception as e:
        return {"ok": False, "error": str(e)}




## 📊 Visualisation Functions

In [3]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    forecast = weather_data.get("forecast", [])
    if not forecast:
        print("No forecast data available.")
        return None

    dates = [day["date"] for day in forecast]
    min_temps = [day["min_temp"] for day in forecast]
    max_temps = [day["max_temp"] for day in forecast]

    fig, ax = plt.subplots()
    ax.plot(dates, min_temps, marker="o", label="Min Temp (°C)")
    ax.plot(dates, max_temps, marker="o", label="Max Temp (°C)")
    ax.set_title(f"Temperature Trend for {weather_data['location']}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()

    if output_type == "figure":
        return fig
    else:
        plt.show()
        return None



In [2]:
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    forecast = weather_data.get("forecast", [])
    if not forecast:
        print("No forecast data available.")
        return None

    dates = [day["date"] for day in forecast]
    precipitation_chances = [day["precipitation_chance"] for day in forecast]

    fig, ax = plt.subplots()
    ax.bar(dates, precipitation_chances, color="skyblue")
    ax.set_title(f"Precipitation Chances for {weather_data['location']}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Chance of Rain (%)")
    ax.set_ylim(0, 100)

    if output_type == "figure":
        return fig
    else:
        plt.show()
        return None



## 🤖 Natural Language Processing

In [4]:
# Define parse_weather_question() and generate_weather_response() here
import re
from datetime import datetime, timedelta

def parse_weather_question(question: str) -> dict:
    """
    Parse a natural language weather question.

    Returns a dict like:
      {
        "topic": "rain" | "temperature" | "wind" | "humidity" | "general",
        "day": 0 (today) | 1 (tomorrow) | 2..4 (weekday offset) | None,
        "raw": original_question
      }
    """
    if not isinstance(question, str) or not question.strip():
        return {"topic": "general", "day": 0, "raw": question}

    q = question.strip().lower()

    # Topic detection (simple keyword approach)
    topic = "general"
    if any(w in q for w in ["rain", "umbrella", "wet", "shower", "storm"]):
        topic = "rain"
    elif any(w in q for w in ["temp", "temperature", "hot", "cold", "warm", "chilly"]):
        topic = "temperature"
    elif any(w in q for w in ["wind", "windy", "breeze", "gale"]):
        topic = "wind"
    elif any(w in q for w in ["humid", "humidity", "moist"]):
        topic = "humidity"

    # Day detection: today / tomorrow / weekday names
    day_offset = None
    if "today" in q:
        day_offset = 0
    elif "tomorrow" in q:
        day_offset = 1
    else:
        weekdays = ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]
        for i, name in enumerate(weekdays):
            if name in q:
                # compute offset from today to the next occurrence of that weekday (0..6)
                today_idx = datetime.today().weekday()  # Monday=0
                target_idx = i
                delta = (target_idx - today_idx) % 7
                day_offset = delta
                break

    # If no explicit day found, default to today (0) for general/temperature/rain
    if day_offset is None:
        day_offset = 0

    return {"topic": topic, "day": int(day_offset), "raw": question}


def generate_weather_response(parsed_question: dict, weather_data: dict) -> str:
    """
    Generate a natural language response based on parsed question and weather data.
    Uses the unified weather_data schema produced by get_weather_data().
    """
    # Basic validation
    if not isinstance(parsed_question, dict):
        return "Sorry, I could not understand the question."
    if not isinstance(weather_data, dict) or (weather_data.get("ok") is False):
        return "Sorry, weather data is not available right now."

    topic = parsed_question.get("topic", "general")
    day = int(parsed_question.get("day", 0))

    location = weather_data.get("location", "your location")
    current = weather_data.get("current", {})
    forecast = weather_data.get("forecast", [])

    # Clamp day index to available forecast range
    if not forecast:
        # Fall back to a simple current-conditions sentence
        temp = current.get("temperature")
        cond = current.get("condition", "unavailable")
        if temp is None:
            return f"Current conditions for {location} are {cond}."
        return f"Right now in {location} it is {temp}°C and {cond}."
    if day < 0:
        day = 0
    if day >= len(forecast):
        day = len(forecast) - 1

    day_data = forecast[day]
    day_label = "today" if day == 0 else ("tomorrow" if day == 1 else f"on {day_data.get('date','that day')}")

    # Topic-specific responses
    if topic == "rain":
        chance = day_data.get("precipitation_chance")
        if chance is None:
            return f"Rain probability {day_label} in {location} is unavailable."
        verdict = "likely" if chance >= 50 else "unlikely"
        return f"Rain is {verdict} {day_label} in {location} (chance {chance}%)."

    elif topic == "temperature":
        tmin = day_data.get("min_temp")
        tmax = day_data.get("max_temp")
        cond = day_data.get("condition", "unknown conditions")
        if tmin is None or tmax is None:
            return f"Temperature details {day_label} in {location} are unavailable."
        return f"{day_label.capitalize()} in {location}: {cond}, {tmin}–{tmax}°C."

    elif topic == "wind":
        # Our forecast schema does not include wind per day; provide current wind as a best-effort.
        ws = current.get("wind_speed")
        wd = current.get("wind_direction", "")
        if ws is None:
            return f"Wind details are not available right now for {location}."
        return f"Current wind in {location} is about {ws} km/h {wd}. Daily wind forecast is not available."

    elif topic == "humidity":
        h = current.get("humidity")
        if h is None:
            return f"Humidity is not available right now for {location}."
        return f"Current humidity in {location} is {h}%."

    # General / fallback response
    tnow = current.get("temperature")
    cnow = current.get("condition", "unknown conditions")
    if tnow is None:
        return f"Current conditions for {location}: {cnow}."
    return f"Right now in {location} it is {tnow}°C and {cnow}. {day_label.capitalize()} looks {day_data.get('condition','unclear')} with {day_data.get('min_temp','?')}–{day_data.get('max_temp','?')}°C."

SyntaxError: invalid syntax (ipython-input-320667227.py, line 12)

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.